In [1]:
!pip install geocoder
!pip install folium

     |████████████████████████████████| 98 kB 6.1 MB/s  eta 0:00:01
     |████████████████████████████████| 94 kB 4.0 MB/s  eta 0:00:01


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [3]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [5]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [6]:
neighborhoods_data = newyork_data['features']

In [7]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [8]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [9]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [10]:
neighborhoods['Borough'].value_counts()

Queens           81
Brooklyn         70
Staten Island    63
Bronx            52
Manhattan        40
Name: Borough, dtype: int64

### Analyzing neighborhoods of Brooklyn

In [13]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [14]:
CLIENT_ID = 'FNKI3B5MFASSIT5L0QSHA5LMBWCIZQUIM4X5RPVW3UKUV45J' # your Foursquare ID
CLIENT_SECRET = 'WVUWFXPWVT4PM0FF0ZZ2IOUHFKH2KL2ZAUCIPZRLYW2EOAZA' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FNKI3B5MFASSIT5L0QSHA5LMBWCIZQUIM4X5RPVW3UKUV45J
CLIENT_SECRET:WVUWFXPWVT4PM0FF0ZZ2IOUHFKH2KL2ZAUCIPZRLYW2EOAZA


In [15]:
brooklyn_data.loc[0, 'Neighborhood']

'Bay Ridge'

In [16]:
neighborhood_latitude = brooklyn_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = brooklyn_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = brooklyn_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bay Ridge are 40.625801065010656, -74.03062069353813.


In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=FNKI3B5MFASSIT5L0QSHA5LMBWCIZQUIM4X5RPVW3UKUV45J&client_secret=WVUWFXPWVT4PM0FF0ZZ2IOUHFKH2KL2ZAUCIPZRLYW2EOAZA&v=20180604&ll=40.625801065010656,-74.03062069353813&radius=500&limit=100'

In [18]:
results = requests.get(url).json()

In [19]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Pilo Arts Day Spa and Salon,Spa,40.624748,-74.030591
1,Bagel Boy,Bagel Shop,40.627896,-74.029335
2,Leo's Casa Calamari,Pizza Place,40.624200,-74.030931
3,Cocoa Grinder,Juice Bar,40.623967,-74.030863
4,Pegasus Cafe,Breakfast Spot,40.623168,-74.031186


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

81 venues were returned by Foursquare.


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude'])

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [24]:
brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,49,49,49,49,49,49
Bay Ridge,81,81,81,81,81,81
Bedford Stuyvesant,30,30,30,30,30,30
Bensonhurst,31,31,31,31,31,31
Bergen Beach,6,6,6,6,6,6
Boerum Hill,89,89,89,89,89,89
Borough Park,20,20,20,20,20,20
Brighton Beach,43,43,43,43,43,43
Broadway Junction,18,18,18,18,18,18


In [25]:
print('There are {} uniques categories.'.format(len(brooklyn_venues['Venue Category'].unique())))

There are 292 uniques categories.


In [26]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Gym,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Sto

In [27]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Gym,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Sto

In [28]:
num_top_venues = 5

for hood in brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                  venue  freq
0    Chinese Restaurant  0.06
1  Fast Food Restaurant  0.04
2   Peruvian Restaurant  0.04
3           Gas Station  0.04
4  Cantonese Restaurant  0.04


----Bay Ridge----
                 venue  freq
0   Italian Restaurant  0.07
1                  Spa  0.06
2          Pizza Place  0.05
3  American Restaurant  0.04
4             Pharmacy  0.04


----Bedford Stuyvesant----
           venue  freq
0    Coffee Shop  0.10
1  Deli / Bodega  0.10
2            Bar  0.07
3           Café  0.07
4    Pizza Place  0.07


----Bensonhurst----
                venue  freq
0  Chinese Restaurant  0.10
1         Pizza Place  0.10
2    Sushi Restaurant  0.06
3  Italian Restaurant  0.06
4          Donut Shop  0.06


----Bergen Beach----
                venue  freq
0     Harbor / Marina  0.33
1  Athletics & Sports  0.17
2                Park  0.17
3      Baseball Field  0.17
4          Playground  0.17


----Boerum Hill----
                    venue  freq
0    

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Chinese Restaurant,Pharmacy,Fast Food Restaurant,Bubble Tea Shop,Cantonese Restaurant,Peruvian Restaurant,Donut Shop,Sushi Restaurant,Gas Station,Italian Restaurant
1,Bay Ridge,Italian Restaurant,Spa,Pizza Place,Greek Restaurant,Chinese Restaurant,Bar,Bagel Shop,Pharmacy,American Restaurant,Gym / Fitness Center
2,Bedford Stuyvesant,Coffee Shop,Deli / Bodega,Café,Pizza Place,Bar,Discount Store,Tiki Bar,Fried Chicken Joint,New American Restaurant,Boutique
3,Bensonhurst,Pizza Place,Chinese Restaurant,Sushi Restaurant,Ice Cream Shop,Donut Shop,Italian Restaurant,Shabu-Shabu Restaurant,Liquor Store,Noodle House,Supermarket
4,Bergen Beach,Harbor / Marina,Athletics & Sports,Baseball Field,Playground,Park,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market
5,Boerum Hill,Coffee Shop,Dance Studio,Bar,French Restaurant,Furniture / Home Store,Bakery,Sandwich Place,Arts & Crafts Store,Cocktail Bar,Cosmetics Shop
6,Borough Park,Bank,Fast Food Restaurant,Pizza Place,Pharmacy,Café,Grocery Store,Optical Shop,Chinese Restaurant,Coffee Shop,Bakery
7,Brighton Beach,Eastern European Restaurant,Russian Restaurant,Restaurant,Pharmacy,Sushi Restaurant,Gourmet Shop,Beach,Bank,Mobile Phone Shop,Coffee Shop
8,Broadway Junction,Gas Station,Donut Shop,Bus Station,Ice Cream Shop,Bus Stop,Seafood Restaurant,Caribbean Restaurant,Nightclub,Breakfast Spot,Sandwich Place
9,Brooklyn Heights,Deli / Bodega,Yoga Studio,Pizza Place,Coffee Shop,Cosmetics Shop,Gym,Park,Italian Restaurant,Bakery,Plaza


### Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [31]:
# set number of clusters
kclusters = 5

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 4, 4, 1, 4, 3, 4, 3, 4], dtype=int32)

In [32]:
brooklyn_merged = brooklyn_data

# add clustering labels
brooklyn_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,4,Italian Restaurant,Spa,Pizza Place,Greek Restaurant,Chinese Restaurant,Bar,Bagel Shop,Pharmacy,American Restaurant,Gym / Fitness Center
1,Brooklyn,Bensonhurst,40.611009,-73.995180,4,Pizza Place,Chinese Restaurant,Sushi Restaurant,Ice Cream Shop,Donut Shop,Italian Restaurant,Shabu-Shabu Restaurant,Liquor Store,Noodle House,Supermarket
2,Brooklyn,Sunset Park,40.645103,-74.010316,4,Pizza Place,Latin American Restaurant,Bank,Bakery,Mexican Restaurant,Fried Chicken Joint,Mobile Phone Shop,Gym,Deli / Bodega,Chinese Restaurant
3,Brooklyn,Greenpoint,40.730201,-73.954241,4,Pizza Place,Bar,Coffee Shop,Cocktail Bar,Grocery Store,French Restaurant,Mexican Restaurant,Deli / Bodega,Record Shop,Yoga Studio
4,Brooklyn,Gravesend,40.595260,-73.973471,1,Pizza Place,Lounge,Spa,Deli / Bodega,Chinese Restaurant,Bakery,Furniture / Home Store,Bar,Sporting Goods Shop,Music Venue


### Examine Clusters

#### Cluster 1

In [34]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,Prospect Park South,Caribbean Restaurant,Fast Food Restaurant,Grocery Store,Pizza Place,Mobile Phone Shop,Mexican Restaurant,Department Store,Donut Shop,Deli / Bodega,Fried Chicken Joint


#### Cluster 2

In [35]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Gravesend,Pizza Place,Lounge,Spa,Deli / Bodega,Chinese Restaurant,Bakery,Furniture / Home Store,Bar,Sporting Goods Shop,Music Venue
17,Bedford Stuyvesant,Coffee Shop,Deli / Bodega,Café,Pizza Place,Bar,Discount Store,Tiki Bar,Fried Chicken Joint,New American Restaurant,Boutique
22,Gowanus,Coffee Shop,Bar,Italian Restaurant,Furniture / Home Store,Art Gallery,Pizza Place,Playground,Food Truck,Deli / Bodega,Gym / Fitness Center
23,Fort Greene,Flower Shop,Italian Restaurant,Wine Shop,Restaurant,Cocktail Bar,Coffee Shop,Playground,Performing Arts Venue,Opera House,French Restaurant
32,Coney Island,Baseball Stadium,Theme Park Ride / Attraction,Skating Rink,Monument / Landmark,Farmers Market,Caribbean Restaurant,Music Venue,Park,Food Stand,Brewery
42,Prospect Lefferts Gardens,Café,Bakery,Pizza Place,Caribbean Restaurant,Deli / Bodega,Wine Shop,Indian Restaurant,Restaurant,Sushi Restaurant,Food
57,Remsen Village,Caribbean Restaurant,Fast Food Restaurant,Sandwich Place,Coffee Shop,Breakfast Spot,Supermarket,Gas Station,Fish Market,Gym,Fried Chicken Joint
60,Mill Basin,Chinese Restaurant,Pizza Place,Japanese Restaurant,Grocery Store,Bagel Shop,Sushi Restaurant,Liquor Store,Bank,Supermarket,Breakfast Spot
65,Dumbo,Park,Bakery,Scenic Lookout,Yoga Studio,Pizza Place,Café,Boxing Gym,Bookstore,Gym,Italian Restaurant


#### Cluster 3

In [36]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 2, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Windsor Terrace,Plaza,Park,Grocery Store,Deli / Bodega,Beer Store,Bar,Middle Eastern Restaurant,French Restaurant,Food Truck,Memorial Site
52,Ocean Parkway,Pizza Place,Sushi Restaurant,Liquor Store,Bagel Shop,Bus Stop,Playground,Steakhouse,General Entertainment,Sake Bar,Gym


#### Cluster 4

In [37]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 3, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Sheepshead Bay,Turkish Restaurant,Dessert Shop,Yoga Studio,Playground,Café,Chinese Restaurant,Sandwich Place,Russian Restaurant,Restaurant,Creperie
8,Flatbush,Pharmacy,Coffee Shop,Caribbean Restaurant,Mexican Restaurant,Juice Bar,Bakery,Middle Eastern Restaurant,Bank,Pizza Place,Donut Shop
24,Park Slope,Coffee Shop,Burger Joint,Bagel Shop,Bakery,Pizza Place,Pet Store,Spa,Mexican Restaurant,Italian Restaurant,Bookstore
25,Cypress Hills,Pizza Place,Fast Food Restaurant,Metro Station,Fried Chicken Joint,Latin American Restaurant,Donut Shop,Ice Cream Shop,Dance Studio,Chinese Restaurant,Food
27,Starrett City,Supermarket,American Restaurant,Pizza Place,Bus Station,Bus Stop,Donut Shop,Pharmacy,Gym Pool,Liquor Store,Shopping Mall
28,Canarsie,Food,Asian Restaurant,Thai Restaurant,Gym,Grocery Store,Caribbean Restaurant,Flea Market,Fish & Chips Shop,Fish Market,Fishing Spot
29,Flatlands,Pharmacy,Fried Chicken Joint,Caribbean Restaurant,Fast Food Restaurant,Park,Nightclub,Lounge,Chinese Restaurant,Bar,Paper / Office Supplies Store
48,Georgetown,Bank,Breakfast Spot,Donut Shop,Pharmacy,Burger Joint,Shopping Mall,Frozen Yogurt Shop,Mexican Restaurant,Miscellaneous Shop,Shipping Store
56,Rugby,Chinese Restaurant,Grocery Store,Caribbean Restaurant,Bank,Fast Food Restaurant,Seafood Restaurant,Gas Station,Supermarket,Pharmacy,Diner
58,New Lots,Chinese Restaurant,Fried Chicken Joint,Pharmacy,Bus Station,Plaza,Grocery Store,Fast Food Restaurant,Breakfast Spot,Park,Bank


#### Cluster 5

In [38]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 4, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Italian Restaurant,Spa,Pizza Place,Greek Restaurant,Chinese Restaurant,Bar,Bagel Shop,Pharmacy,American Restaurant,Gym / Fitness Center
1,Bensonhurst,Pizza Place,Chinese Restaurant,Sushi Restaurant,Ice Cream Shop,Donut Shop,Italian Restaurant,Shabu-Shabu Restaurant,Liquor Store,Noodle House,Supermarket
2,Sunset Park,Pizza Place,Latin American Restaurant,Bank,Bakery,Mexican Restaurant,Fried Chicken Joint,Mobile Phone Shop,Gym,Deli / Bodega,Chinese Restaurant
3,Greenpoint,Pizza Place,Bar,Coffee Shop,Cocktail Bar,Grocery Store,French Restaurant,Mexican Restaurant,Deli / Bodega,Record Shop,Yoga Studio
5,Brighton Beach,Eastern European Restaurant,Russian Restaurant,Restaurant,Pharmacy,Sushi Restaurant,Gourmet Shop,Beach,Bank,Mobile Phone Shop,Coffee Shop
7,Manhattan Terrace,Pizza Place,Grocery Store,Donut Shop,Ice Cream Shop,Organic Grocery,Coffee Shop,Convenience Store,Cosmetics Shop,Steakhouse,Bank
9,Crown Heights,Pizza Place,Café,Museum,Farmers Market,Coffee Shop,Bakery,Bagel Shop,Liquor Store,Supermarket,Candy Store
10,East Flatbush,Chinese Restaurant,Caribbean Restaurant,Wine Shop,Liquor Store,Department Store,Food & Drink Shop,Supermarket,Pharmacy,Park,Fishing Spot
11,Kensington,Thai Restaurant,Grocery Store,Pizza Place,Ice Cream Shop,Sandwich Place,Furniture / Home Store,Coffee Shop,Gas Station,Liquor Store,Supermarket
13,Prospect Heights,Bar,Mexican Restaurant,Wine Shop,Thai Restaurant,Bakery,Coffee Shop,Cocktail Bar,Deli / Bodega,Ice Cream Shop,New American Restaurant
